# Python для анализа данных

## Постановка скриптов на расписание

*Автор: Ян Пиле, НИУ ВШЭ*

Думаю, многим из вас хотелось бы автоматизировать какие-то рутинные процессы в быту или на работе, например, сборку одинакового вида отчетов, подгрузку актуальных данных о курсе валют в вашу базу данных или сводку курсов акций на сегодня. Для этого вам необходимо написать код, который решает задачу, а потом заставить его выполняться с некоторой периодичностью. В Python для этого существует механизм "постановки скриптов на расписание" – в прямом смысле слова, по часам.

В Unix-подобных системах такую штуку решают с помощью [CRON](https://ru.wikipedia.org/wiki/Cron), а в Python, как всегда, придумали человеческий интерфейс :) Представляю вм библиотеку schedule. Поехали!

In [2]:
!pip install schedule

In [75]:
import schedule 
import time

def greet(name):
    print('Hello', name)

schedule.every(2).seconds.do(greet, name='Alice').tag('my_shiny_job')

Every 2 seconds do greet(name='Alice') (last run: [never], next run: 2020-10-07 09:39:46)

Чтобы завести какой-то процесс, надо запустить его в цикле такого вида:

In [52]:
while True:
    schedule.run_pending()
    time.sleep(1)
    

Hello Alice
Hello Alice


KeyboardInterrupt: 

Как посмотреть список job'ов?

In [76]:
schedule.jobs

[Every 2 seconds do greet(name='Alice') (last run: [never], next run: 2020-10-07 09:39:46)]

Как прекратить job?

In [78]:
schedule.clear('my_shiny_job')
schedule.jobs

[]

Варианты постановок на расписание могут быть всякие:

In [30]:
def job():
    print("Работаю")

In [13]:
schedule.every(10).minutes.do(job)
schedule.every().hour.do(job)
schedule.every().day.at("10:30").do(job)

# Отдельно стоит упомянуть вот это – это запуск со случайным интервалом от 5 до 10 минут.
schedule.every(5).to(10).minutes.do(job)


schedule.every().monday.do(job)
schedule.every().wednesday.at("13:15").do(job)
schedule.every().minute.at(":17").do(job)

Every 1 minute at 00:00:17 do job() (last run: [never], next run: 2020-10-06 23:15:17)

**ВАЖНО:** Библиотека сама не умеет обрабатывать исключения в задачах, поэтому, возможно, понадобится  декоратор, который будет отменять работу, если произошло исключение.

In [27]:
import functools

# декоратор для отлова исключений
def catch_exceptions(cancel_on_failure=False):
    
    def catch_exceptions_decorator(job_func):
        @functools.wraps(job_func)
        def wrapper(*args, **kwargs):
            try:
                return job_func(*args, **kwargs)
            except:
                import traceback
                print(traceback.format_exc())
                if cancel_on_failure:
                    return schedule.CancelJob
        return wrapper
    
    return catch_exceptions_decorator


@catch_exceptions(cancel_on_failure=True)
def bad_task():
    # даст исключение, но декоратор отменяет эту задачу
    return 1 / 0  

schedule.every(5).seconds.do(bad_task)

Every 5 seconds do bad_task() (last run: [never], next run: 2020-10-06 23:21:54)

Как почистить список job'ов?

In [74]:
schedule.clear()

In [28]:
while True:
    schedule.run_pending()
    time.sleep(1)

Traceback (most recent call last):
  File "<ipython-input-27-062bcafb0c44>", line 10, in wrapper
    return job_func(*args, **kwargs)
  File "<ipython-input-27-062bcafb0c44>", line 24, in bad_task
    return 1 / 0
ZeroDivisionError: division by zero



KeyboardInterrupt: 

Если нужно отменить группу заданий, то к ним добавляют тэги:

In [31]:
schedule.every().day.do(greet, 'Monica').tag('daily-tasks')
schedule.every().day.do(greet, 'Derek').tag('daily-tasks')
schedule.clear('daily-tasks')  # массовая отмена по тэгу

А вот и [документация](https://schedule.readthedocs.io/en/stable/faq.html#how-to-run-a-job-at-random-intervals) по scheduler'у 

Это не единственный способ постановки чего бы то ни было на расписание. Например мне в работе приходилось использовать библиотеку [python-crontab](https://stackabuse.com/scheduling-jobs-with-python-crontab/). 

А если вы любите Windows, вы можете использовать [нативный постановщик на расписание](https://www.jcchouinard.com/python-automation-using-task-scheduler/) в формате:
    1) Написать скрипт в отдельный файл с форматом .py
    2) Найти этот скрипт в интерфейсе Task Scheduler'a
    3) Поставить на расписание.
    
Я так не делал никогда, так что вы можете стать первопроходцами :)

Ну и в конце СУПЕРполезная ссылка. Во многих корпорациях в качестве планировщика задач используют Apache Airflow. И так случилось, что это Open-source ПО и вы можете взять и установить его себе. Так что рекомендую [попробовать](https://airflow.apache.org)